In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
import pandas as pd

In [2]:
train_dir = r'C:\Users\lol51\Downloads\OCT_data\train'
test_dir = r'C:\Users\lol51\Downloads\OCT_data\test'
val_dir = r'C:\Users\lol51\Downloads\OCT_data\val'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

Found 10000 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [5]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(4, activation='softmax')  # 4 classes: NORMAL, CNV, DME, DRUSEN
])

In [9]:
base_model.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
313/313 [==============================] - 399s 1s/step - loss: 0.3076 - accuracy: 0.9507 - val_loss: 0.0803 - val_accuracy: 0.9800
Epoch 2/10
313/313 [==============================] - 395s 1s/step - loss: 0.0744 - accuracy: 0.9761 - val_loss: 0.2733 - val_accuracy: 0.9400
Epoch 3/10
313/313 [==============================] - 330s 1s/step - loss: 0.0506 - accuracy: 0.9828 - val_loss: 0.1755 - val_accuracy: 0.9700
Epoch 4/10
313/313 [==============================] - 319s 1s/step - loss: 0.0417 - accuracy: 0.9864 - val_loss: 0.0768 - val_accuracy: 0.9800
Epoch 5/10
313/313 [==============================] - 307s 980ms/step - loss: 0.0202 - accuracy: 0.9939 - val_loss: 0.1599 - val_accuracy: 0.9800
Epoch 6/10
313/313 [==============================] - 309s 988ms/step - loss: 0.0221 - accuracy: 0.9916 - val_loss: 0.1378 - val_accuracy: 0.9700
Epoch 7/10
313/313 [==============================] - 318s 1s/step - loss: 0.0199 - accuracy: 0.9946 - val_loss: 0.1856 - val_accuracy: 

In [7]:
base_model.trainable = True
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy']) 

In [8]:
model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10


313/313 [==============================] - 1392s 4s/step - loss: 0.4755 - accuracy: 0.8270 - val_loss: 0.1946 - val_accuracy: 0.9200
Epoch 2/10
313/313 [==============================] - 1373s 4s/step - loss: 0.2359 - accuracy: 0.9197 - val_loss: 0.0780 - val_accuracy: 0.9900
Epoch 3/10
313/313 [==============================] - 1357s 4s/step - loss: 0.1801 - accuracy: 0.9373 - val_loss: 0.1273 - val_accuracy: 0.9600
Epoch 4/10
313/313 [==============================] - 1528s 5s/step - loss: 0.1301 - accuracy: 0.9556 - val_loss: 0.1297 - val_accuracy: 0.9500
Epoch 5/10
313/313 [==============================] - 1428s 5s/step - loss: 0.1015 - accuracy: 0.9659 - val_loss: 0.1963 - val_accuracy: 0.9400
Epoch 6/10
313/313 [==============================] - 1532s 5s/step - loss: 0.0807 - accuracy: 0.9744 - val_loss: 0.1098 - val_accuracy: 0.9500
Epoch 7/10
313/313 [==============================] - 1543s 5s/step - loss: 0.0557 - accuracy: 0.9827 - val_loss: 0.0656 - val_accurac

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=32, class_mode=None, shuffle=False)
predictions = model.predict(test_generator)

Found 200 images belonging to 1 classes.
7/7 [==============================] - 7s 893ms/step


In [12]:
predicted_labels = tf.argmax(predictions, axis=-1).numpy()

In [14]:
df = pd.DataFrame({'filename': test_generator.filenames, 'label': predicted_labels})
df.to_csv('predictions1.csv', index=False)

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [16]:
true_labels = test_generator.classes

In [18]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

C:\Users\lol51\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.28
Precision: 0.25
Recall: 0.07
F1 Score: 0.10937500000000001
